In [25]:
import pandas as pd
import numpy as np
data_divar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/divar_posts_dataset.csv')

In [26]:
data_divar.head(2)

,Unnamed: 0,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year
0,0,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,کلاسیک و شیک و استثنایی\nچرم مالزی\nچوب راش\nف...,54761638662241,5,NaN,mobile,3850000,ست مبلمان و نهارخوری ٩ نفره,NaN,NaN
1,1,False,NaN,for-the-home,furniture-and-home-decore,antiques-and-art,Mashhad,Tuesday 07PM,"سلام,یک عدد گلدون نخل سه طبقه ی سالم دارم با پ...",42727631379133,1,NaN,mobile,30000,گلدون مصنوعی نخل,NaN,NaN


In [27]:
data_divar['product']=data_divar.apply(lambda x:'%s_%s_%s' % (x['cat1'],x['cat2'],x['cat3']),axis=1)


In [28]:
data_divar['Q']=1

In [30]:
data_divar=data_divar[['city','product','Q']]
data_divar.head()

,city,product,Q
0,Tehran,for-the-home_furniture-and-home-decore_sofa-ar...,1
1,Mashhad,for-the-home_furniture-and-home-decore_antique...,1
2,Mashhad,vehicles_cars_heavy,1
3,Tehran,for-the-home_furniture-and-home-decore_sofa-ar...,1
4,Karaj,personal_baby-and-toys_personal-toys,1


In [40]:
pip install fpgrowth_py

In [42]:
pip install pyfpgrowth

     |████████████████████████████████| 1.6MB 8.7MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=81f13e93023e8af233771f668fe36bfc0e55414324c35680d95cfefb0fb49892
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [46]:
def unique(list1):
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [63]:
grouped = data_divar.groupby(by="product")
transactions = []
grouped.filter(lambda x: (transactions.append(unique(x['city'].to_list())) if x.shape[0] >= 2 else True))
transactions[0:5]

[['Tehran',
  'Shiraz',
  'Mashhad',
  'Isfahan',
  'Karaj',
  'Tabriz',
  'Ahvaz',
  'Kermanshah',
  'Qom'],
 ['Mashhad', 'Tehran', 'Karaj'],
 ['Isfahan', 'Tehran', 'Karaj', 'Mashhad', 'Shiraz'],
 ['Tabriz',
  'Tehran',
  'Isfahan',
  'Shiraz',
  'Karaj',
  'Qom',
  'Mashhad',
  'Kermanshah',
  'Ahvaz'],
 ['Tehran']]

In [66]:
import pyfpgrowth
from fpgrowth_py import fpgrowth

patterns = pyfpgrowth.find_frequent_patterns(transactions, 50)
rules = pyfpgrowth. generate_association_rules(patterns,0.99)
print('frequent patterns:')
print(patterns)
print('rules:')
rules

frequent patterns:
{('Shiraz',): 51, ('Mashhad', 'Shiraz'): 51, ('Shiraz', 'Tehran'): 51, ('Mashhad', 'Shiraz', 'Tehran'): 51, ('Karaj',): 52, ('Karaj', 'Mashhad'): 52, ('Karaj', 'Tehran'): 52, ('Karaj', 'Mashhad', 'Tehran'): 52, ('Mashhad',): 59, ('Mashhad', 'Tehran'): 59, ('Tehran',): 87}
rules:


{('Karaj',): (('Mashhad', 'Tehran'), 1.0),
 ('Karaj', 'Mashhad'): (('Tehran',), 1.0),
 ('Karaj', 'Tehran'): (('Mashhad',), 1.0),
 ('Mashhad',): (('Tehran',), 1.0),
 ('Mashhad', 'Shiraz'): (('Tehran',), 1.0),
 ('Shiraz',): (('Mashhad', 'Tehran'), 1.0),
 ('Shiraz', 'Tehran'): (('Mashhad',), 1.0)}

In [ ]:
basket = (data_divar.groupby(['city', 'product'])['Q']
          .sum().unstack().reset_index().fillna(0)
          .set_index('city'))
basket

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(basket_sets, min_support=0.88, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()